### Сверточные нейронные сети

##### 1. обучить рукописную сверточную нейронную сеть (с падением размера ядра свертки и последовательностью блоков свертка-пулинг (conv-pool)-(conv-pool)-...) на датасете cifar-10 
##### 2. оценить рост точности при увеличении ширины сети (больше фильтров)
##### 3. оценить рост точности при увеличении глубины сети (больше слоев)
##### 4. *сравнить с точностью полносвязной сети для этой выборки

In [44]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.optimizers import SGD
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import os

In [40]:
classes = ['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']

In [41]:
# установка параметров нейросети
batch_size = 32
num_classes = 10
epochs = 20
data_augmentation = False
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'тренировочные примеры')
print(X_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


X_train = X_train / 255.0
X_test = X_test / 255.0

X_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


##### 4-х слойная модель фильтр (3x3)

In [24]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_26 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_27 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 15, 15, 64)       

In [25]:
# Обучаем модель
hist = model.fit(X_train, y_train,
                 epochs=epochs,
                 batch_size=batch_size,
                 validation_data=(X_test, y_test),
                 verbose=1)

test_loss, test_metric = model.evaluate(X_test, y_test)
print(f"Test loss {test_loss:.4f}, accuracy {test_metric:.2f}")

Epoch 1/20
1563/1563 [==============================] - 149s 95ms/step - loss: 2.2094 - accuracy: 0.1590 - val_loss: 1.9041 - val_accuracy: 0.3248
Epoch 2/20
1563/1563 [==============================] - 149s 95ms/step - loss: 1.9104 - accuracy: 0.3060 - val_loss: 1.6214 - val_accuracy: 0.4070
Epoch 3/20
1563/1563 [==============================] - 155s 99ms/step - loss: 1.6815 - accuracy: 0.3901 - val_loss: 1.6062 - val_accuracy: 0.4257
Epoch 4/20
1563/1563 [==============================] - 160s 102ms/step - loss: 1.5290 - accuracy: 0.4396 - val_loss: 1.4126 - val_accuracy: 0.4936
Epoch 5/20
1563/1563 [==============================] - 153s 98ms/step - loss: 1.4294 - accuracy: 0.4791 - val_loss: 1.3174 - val_accuracy: 0.5294
Epoch 6/20
1563/1563 [==============================] - 150s 96ms/step - loss: 1.3479 - accuracy: 0.5120 - val_loss: 1.2332 - val_accuracy: 0.5577
Epoch 7/20
1563/1563 [==============================] - 141s 91ms/step - loss: 1.2737 - accuracy: 0.5443 - val_loss: 

##### 4-х слойная модель фильтр (5x5)

In [26]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (5, 5), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(32, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (5, 5), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 32, 32, 32)        2432      
_________________________________________________________________
activation_32 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 28, 28, 32)        25632     
_________________________________________________________________
activation_33 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 14, 14, 64)       

In [27]:
# Обучаем модель
hist = model.fit(X_train, y_train,
                 epochs=epochs,
                 batch_size=batch_size,
                 validation_data=(X_test, y_test),
                 verbose=1)

test_loss, test_metric = model.evaluate(X_test, y_test)
print(f"Test loss {test_loss:.4f}, accuracy {test_metric:.2f}")

Epoch 1/20
1563/1563 [==============================] - 234s 149ms/step - loss: 2.1877 - accuracy: 0.1673 - val_loss: 1.7761 - val_accuracy: 0.3568
Epoch 2/20
1563/1563 [==============================] - 233s 149ms/step - loss: 1.7896 - accuracy: 0.3402 - val_loss: 1.5239 - val_accuracy: 0.4395
Epoch 3/20
1563/1563 [==============================] - 239s 153ms/step - loss: 1.5879 - accuracy: 0.4154 - val_loss: 1.3882 - val_accuracy: 0.4974
Epoch 4/20
1563/1563 [==============================] - 233s 149ms/step - loss: 1.4493 - accuracy: 0.4724 - val_loss: 1.3310 - val_accuracy: 0.5268
Epoch 5/20
1563/1563 [==============================] - 229s 147ms/step - loss: 1.3532 - accuracy: 0.5151 - val_loss: 1.2077 - val_accuracy: 0.5687
Epoch 6/20
1563/1563 [==============================] - 229s 147ms/step - loss: 1.2748 - accuracy: 0.5423 - val_loss: 1.1497 - val_accuracy: 0.5889
Epoch 7/20
1563/1563 [==============================] - 229s 147ms/step - loss: 1.1999 - accuracy: 0.5717 - val_

##### 6-и слойная модель

In [45]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=X_train.shape[1:]))
model.add(Dropout(0.2))

model.add(Conv2D(32,(3,3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model.add(Dropout(0.2))

model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024,activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_23 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_24 (Dropout)         (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 16, 16, 64)      

In [46]:
# Обучаем модель
hist = model.fit(X_train, y_train,
                 epochs=epochs,
                 batch_size=batch_size,
                 validation_data=(X_test, y_test),
                 verbose=1)

test_loss, test_metric = model.evaluate(X_test, y_test)
print(f"Test loss {test_loss:.4f}, accuracy {test_metric:.2f}")

Epoch 1/20
1563/1563 [==============================] - 283s 181ms/step - loss: 2.2166 - accuracy: 0.1588 - val_loss: 1.8797 - val_accuracy: 0.3528
Epoch 2/20
1563/1563 [==============================] - 277s 177ms/step - loss: 1.8269 - accuracy: 0.3378 - val_loss: 1.6578 - val_accuracy: 0.4175
Epoch 3/20
1563/1563 [==============================] - 274s 175ms/step - loss: 1.6033 - accuracy: 0.4171 - val_loss: 1.5254 - val_accuracy: 0.4586
Epoch 4/20
1563/1563 [==============================] - 287s 184ms/step - loss: 1.4778 - accuracy: 0.4658 - val_loss: 1.3990 - val_accuracy: 0.5114
Epoch 5/20
1563/1563 [==============================] - 276s 176ms/step - loss: 1.3969 - accuracy: 0.4971 - val_loss: 1.3900 - val_accuracy: 0.5108
Epoch 6/20
1563/1563 [==============================] - 290s 185ms/step - loss: 1.3323 - accuracy: 0.5211 - val_loss: 1.2461 - val_accuracy: 0.5553
Epoch 7/20
1563/1563 [==============================] - 267s 171ms/step - loss: 1.2540 - accuracy: 0.5517 - val_

##### Полносвязная модель

In [52]:
input = Input(shape=(32, 32))
flat = Flatten()(input)

x1 = Dense(128, Activation('relu'))(flat)
x1 = Dense(10, Activation('softmax'))(x1)
model1 = Model(inputs=input, outputs=x1)
# не обучаем слои модели
for layer in model1.layers:
    layer.trainable = False


x2 = Dense(256, Activation('relu'))(flat)
x2 = Dense(10, Activation('softmax'))(x2)
model2 = Model(inputs=input, outputs=x2)
# не обучаем слои модели
for layer in model2.layers:
    layer.trainable = False


x3 = Dense(256, activation='relu')(flat)
x3 = Dense(10, activation='softmax')(x3)
model3 = Model(inputs=input, outputs=x3)
# не обучаем слои модели
for layer in model3.layers:
    layer.trainable = False


# усредняем выходы сетей
out_all = tf.keras.layers.Average()([model1.output, model2.output, model3.output])
# добавляем новый обучаемый слой
# out_all = Dense(10, activation='relu')(out_all)

out_all = tf.keras.layers.Softmax()(out_all)

model_all = Model(inputs=[input], outputs=out_all)
model_all.compile(optimizer='SGD',
                  loss=SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

model_all.summary()

Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 32, 32)]     0                                            
__________________________________________________________________________________________________
flatten_15 (Flatten)            (None, 1024)         0           input_8[0][0]                    
__________________________________________________________________________________________________
dense_39 (Dense)                (None, 128)          131200      flatten_15[0][0]                 
__________________________________________________________________________________________________
dense_41 (Dense)                (None, 256)          262400      flatten_15[0][0]                 
___________________________________________________________________________________________

In [53]:
# Обучаем модель
hist = model.fit(X_train, y_train,
                 epochs=epochs,
                 batch_size=batch_size,
                 validation_data=(X_test, y_test),
                 verbose=1)

test_loss, test_metric = model.evaluate(X_test, y_test)
print(f"Test loss {test_loss:.4f}, accuracy {test_metric:.2f}")

Epoch 1/20
1563/1563 [==============================] - 278s 178ms/step - loss: 0.6500 - accuracy: 0.7719 - val_loss: 0.7762 - val_accuracy: 0.7326
Epoch 2/20
1563/1563 [==============================] - 282s 181ms/step - loss: 0.6232 - accuracy: 0.7799 - val_loss: 0.7393 - val_accuracy: 0.7426
Epoch 3/20
1563/1563 [==============================] - 267s 171ms/step - loss: 0.5939 - accuracy: 0.7925 - val_loss: 0.7326 - val_accuracy: 0.7510
Epoch 4/20
1563/1563 [==============================] - 276s 177ms/step - loss: 0.5726 - accuracy: 0.7983 - val_loss: 0.7037 - val_accuracy: 0.7586
Epoch 5/20
1563/1563 [==============================] - 293s 187ms/step - loss: 0.5418 - accuracy: 0.8084 - val_loss: 0.7152 - val_accuracy: 0.7540
Epoch 6/20
1563/1563 [==============================] - 291s 186ms/step - loss: 0.5150 - accuracy: 0.8168 - val_loss: 0.7085 - val_accuracy: 0.7597
Epoch 7/20
1563/1563 [==============================] - 287s 184ms/step - loss: 0.4933 - accuracy: 0.8262 - val_

In [54]:
from prettytable import PrettyTable
import pandas as pd

data = [['4layerCNN(3x3)',72.81,73.00], ['4layerCNN(5x5)',75.64,76.00], ['6layerCNN(3x3)',72.83,73.00], 
        ['Полносвязная модель',78.31,78.00]]
df = pd.DataFrame(data, columns=['Описание эксперимента', 'accuracy на обучении,%', 'accuracy на тесте,%'])

def generate_ascii_table(df):
    x = PrettyTable()
    x.field_names = df.columns.tolist()
    for row in df.values:
        x.add_row(row)
    print(x)
    return x

generate_ascii_table(df)

+-----------------------+------------------------+---------------------+
| Описание эксперимента | accuracy на обучении,% | accuracy на тесте,% |
+-----------------------+------------------------+---------------------+
|     4layerCNN(3x3)    |         72.81          |         73.0        |
|     4layerCNN(5x5)    |         75.64          |         76.0        |
|     6layerCNN(3x3)    |         72.83          |         73.0        |
|  Полносвязная модель  |         78.31          |         78.0        |
+-----------------------+------------------------+---------------------+


##### Вывод: Увеличение ширины сети точность повышается, увеличение глубины сети не улучшает точность. Полносвязная сеть точность увеличивает